In [1]:
!git clone https://github.com/MohamedKhalifa1/Stack-Overflow-Annual-Developer-Survey-Analysis

Cloning into 'Stack-Overflow-Annual-Developer-Survey-Analysis'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 67 (delta 4), reused 16 (delta 1), pack-reused 47
Receiving objects: 100% (67/67), 29.85 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
!unzip /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/preprocessed/01_preprocessed_data.zip

Archive:  /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/preprocessed/01_preprocessed_data.zip
  inflating: content/01_preprocessed_data.pkl  


In [8]:
DATA_PATH = '/content/content/01_preprocessed_data.pkl'
ROLES_PATH = '/content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/raw/roles_short_names.csv'

In [4]:
ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith']

In [5]:
# !unzip /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/raw/stack-overflow-developer-survey-2023.zip -d /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/raw

In [6]:
%cd /content/Stack-Overflow-Annual-Developer-Survey-Analysis

/content/Stack-Overflow-Annual-Developer-Survey-Analysis


In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read data and preprocess

In [9]:
raw_df  = pd.read_pickle(DATA_PATH)
roles_names = pd.read_csv(ROLES_PATH , sep=';')

In [10]:
raw_df.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,[],NaN,[],NaN,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"[Employed, full-time]",Remote,"[Hobby, Contribute to open-source projects, Bo...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Books / Physical media, Colleague, Friend or ...",[Formal documentation provided by the owner of...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,"[DevOps function, Microservices, Automated tes...","Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"[Employed, full-time]","Hybrid (some remote, some in-person)","[Hobby, Professional development or self-paced...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Books / Physical media, Colleague, On the job...",[Formal documentation provided by the owner of...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,"[DevOps function, Microservices, Automated tes...","Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"[Employed, full-time]","Hybrid (some remote, some in-person)",[Hobby],"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Colleague, Friend or family member, Other onl...",[Formal documentation provided by the owner of...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,"[Automated testing, Continuous integration (CI...",NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"[Employed, full-time, Independent contractor, ...",Remote,"[Hobby, Contribute to open-source projects, Pr...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Books / Physical media, Online Courses or Cer...",[Formal documentation provided by the owner of...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,"[Microservices, Automated testing, Observabili...",Other,Appropriate in length,Neither easy nor difficult,23456.0


In [11]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           89184 non-null  int64  
 1   Q120                                 89184 non-null  object 
 2   MainBranch                           89184 non-null  object 
 3   Age                                  89184 non-null  object 
 4   Employment                           89184 non-null  object 
 5   RemoteWork                           73810 non-null  object 
 6   CodingActivities                     89184 non-null  object 
 7   EdLevel                              87973 non-null  object 
 8   LearnCode                            89184 non-null  object 
 9   LearnCodeOnline                      89184 non-null  object 
 10  LearnCodeCoursesCert                 89184 non-null  object 
 11  YearsCode                   

In [12]:
raw_df.describe()

,ResponseId,YearsCode,YearsCodePro,CompTotal,WorkExp,ConvertedCompYearly
count,89184.000000,87435.000000,66136.000000,4.822500e+04,43579.000000,4.801900e+04
mean,44592.500000,13.977926,11.266194,1.036807e+42,11.405126,1.031101e+05
std,25745.347541,10.538807,9.211307,2.276847e+44,9.051989,6.814188e+05
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000,1.000000e+00
25%,22296.750000,6.000000,4.000000,6.300000e+04,5.000000,4.390700e+04
50%,44592.500000,11.000000,9.000000,1.150000e+05,9.000000,7.496300e+04
75%,66888.250000,20.000000,16.000000,2.300000e+05,16.000000,1.216410e+05
max,89184.000000,51.000000,51.000000,5.000000e+46,50.000000,7.435143e+07


In [13]:
roles_names.head()

,Original name,Short name
0,"Developer, back-end",Back-end dev
1,"Developer, full-stack",Full-stack dev
2,"Developer, front-end",Front-end dev
3,"Developer, desktop or enterprise applications",Desktop dev
4,"Developer, mobile",Mobile dev


In [42]:
print(raw_df['DevType'].unique())

[nan 'Senior Executive (C-Suite, VP, etc.)' 'Developer, back-end'
 'Developer, front-end' 'Developer, full-stack' 'System administrator'
 'Developer, desktop or enterprise applications' 'Developer, QA or test'
 'Designer' 'Data scientist or machine learning specialist'
 'Data or business analyst' 'Security professional' 'Educator'
 'Research & Development role' 'Other (please specify):'
 'Developer, mobile' 'Database administrator'
 'Developer, embedded applications or devices' 'Student' 'Engineer, data'
 'Hardware Engineer' 'Product manager' 'Academic researcher'
 'Developer, game or graphics' 'Cloud infrastructure engineer'
 'Engineering manager' 'Developer Experience' 'Project manager'
 'DevOps specialist' 'Engineer, site reliability' 'Blockchain'
 'Developer Advocate' 'Scientist' 'Marketing or sales professional']


In [43]:
print(roles_names['Short name '].unique())

['Back-end dev' 'Full-stack dev' 'Front-end dev' 'Desktop dev'
 'Mobile dev' 'DevOps' 'Database admin' 'Designer' 'System admin'
 'Embedded dev' 'Analyst' 'Data Scientist ' 'Quality Assurance'
 'Data Engineer' 'Researcher' 'Educator' 'Game dev' 'Engineering manager'
 'Product manager' 'Scientist' 'Site Engineer' 'Senior Executive'
 'Marketing/Sales' 'Student' 'Other']


In [14]:
roles_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Original name  25 non-null     object
 1   Short name     25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes


In [23]:
df = raw_df.copy()
encoded_dfs = {}
for col in ROLE_COLS + TECH_COLS:
    df[col].fillna('', inplace=True)

    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [x] if pd.notnull(x) else [])

    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                               columns=binarizer.classes_,
                               index=df.index)
    encoded_dfs[col] = encoded_df

In [24]:
df = pd.concat(encoded_dfs, axis=1)

In [25]:
df.head()

DevType                                                               \
          Academic researcher Blockchain Cloud infrastructure engineer   
0       1                   0          0                             0   
1       0                   0          0                             0   
2       0                   0          0                             0   
3       0                   0          0                             0   
4       0                   0          0                             0   

                                                                          \
  Data or business analyst Data scientist or machine learning specialist   
0                        0                                             0   
1                        0                                             0   
2                        0                                             0   
3                        0                                             0   
4                        0                                             0   

                                                                        ...  \
  Database administrator Designer DevOps specialist Developer Advocate  ...   
0                      0        0                 0                  0  ...   
1                      0        0                 0                  0  ...   
2                      0        0                 0                  0  ...   
3                      0        0                 0                  0  ...   
4                      0        0                 0                  0  ...   

  NEWCollabToolsHaveWorkedWith                                     \
                        Spyder Sublime Text TextMate VSCodium Vim   
0                            0            0        0        0   0   
1                            0            0        0        0   1   
2                            0            0        0        0   0   
3                            0            0        0        0   1   
4                            0            0        0        0   1   

                                                         
  Visual Studio Visual Studio Code WebStorm Xcode condo  
0             0                  0        0     0     0  
1             0                  1        0     0     0  
2             0                  0        0     0     0  
3             0                  1        1     0     0  
4             0                  1        0     0     0  

[5 rows x 304 columns]

In [32]:
jobs_freq = df['DevType'].copy().sum().sort_values().reset_index()
jobs_freq

,index,0
0,Marketing or sales professional,149
1,Developer Advocate,212
2,Database administrator,257
3,Designer,281
4,Hardware Engineer,286
5,Blockchain,323
6,Developer Experience,326
7,Scientist,351
8,Educator,415
9,"Engineer, site reliability",427


# Display Skills Frequancy

In [58]:
skills_freq = df.copy().drop('DevType', axis=1).sum().sort_values(ascending=False).reset_index()
skills_freq.columns = ['group', 'skill', 'freq']

In [62]:
skills_freq

,group,skill,freq
0,NEWCollabToolsHaveWorkedWith,Visual Studio Code,63793
1,LanguageHaveWorkedWith,JavaScript,55711
2,LanguageHaveWorkedWith,HTML/CSS,46396
3,LanguageHaveWorkedWith,Python,43158
4,LanguageHaveWorkedWith,SQL,42623
...,...,...,...
265,ToolsTechHaveWorkedWith,snitch,55
266,ToolsTechHaveWorkedWith,lest,55
267,ToolsTechHaveWorkedWith,CUTE,54
268,ToolsTechHaveWorkedWith,ELFspy,51


In [64]:
fig = px.treemap(skills_freq)

In [ ]:

fig = px.treemap(skills_freq,
                 title='Skills Frequancy',
                 path=['group', 'skill'],
                 color_continuous_scale='deep',
                 values='freq',color='freq')

fig.update_layout(width=1600, height=700)
fig.show()